In [ ]:
! pip install -r "requirements.txt"

In [5]:
import cv2
import keyboard
from time import time


class CodeArea:
    def __init__(self):
        # Load the face cascade
        self.face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

        # Initialize variables
        self.cap = cv2.VideoCapture(0)
        self.start_time = None  # Start time of the active session
        self.is_running = False  # Indicates whether the timer is running
        self.face_detected = False  # Tracks if a face is currently detected
        self.pause_start_time = 0  # Time when the pause started
        self.elapsed_time_before_pause = 0  # Total elapsed time before pause
        self.last_check_time = time()
        self.wait_time_face_detected = 10  # Time interval for checking face detection
        self.wait_time_face_not_detected = 5  # Time interval for checking face absence
        self.manual_pause = False  # Tracks if the system is manually paused
        self.current_time = None
        self.elapsed_pause_time = 0

    def save_prev_time(self, elapsed_time):
        with open('prevoius_elapsed_time.txt','w') as time_record:
            time_record.write(str(elapsed_time))
    
    def fetch_prev_time(self):
            try:
                with open('prevoius_elapsed_time.txt', 'r') as time_fetch:
                    elapsed_time = float(time_fetch.readline().strip())
                    return elapsed_time
            except:
                return 0
            
    def load_timer_state(self):
        prev_time = self.fetch_prev_time()
        if prev_time > 0:
            self.start_time = time() 
            self.is_running = True
            return prev_time
        else: 
            pass

    def start_scan(self):
        while True:
            # If not manually paused, capture video frames
            if not self.manual_pause:
                ret, img = self.cap.read()
                if not ret:
                    print("Failed to capture image.")
                    break

                # Convert frame to grayscale for face detection
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = []

                # Check time elapsed since the last face check
                self.current_time = time()
                elapsed_since_last_check = self.current_time - self.last_check_time

                # Perform face detection based on defined intervals
                if (self.face_detected and elapsed_since_last_check >= self.wait_time_face_detected) or \
                        (not self.face_detected and elapsed_since_last_check >= self.wait_time_face_not_detected):

                    self.last_check_time = self.current_time
                    faces = self.face_cascade.detectMultiScale(gray, 1.1, 4)

                    if len(faces) > 0:
                        """Face Detected: Resume or Start Timer"""
                        if not self.is_running:
                            if self.start_time is None:
                                # First-time initialization of the timer
                                self.start_time = time()
                            else:
                                # Resume from pause
                                self.elapsed_pause_time += time() - self.pause_start_time
                            self.is_running = True  # Start the timer
                        self.face_detected = True
                    else:
                        """No Face Detected: Pause Timer"""
                        if self.is_running:
                            # Save the elapsed active time before pausing
                            self.elapsed_time_before_pause += time() - self.start_time
                            self.is_running = False  # Stop the timer
                        self.face_detected = False
                        self.pause_start_time = time()  # Record when the pause starts

                # Update and display the timer on the video frame
                if self.is_running:
                    # Calculate total elapsed time only when running
                    elapsed_time = time() - self.start_time + self.elapsed_time_before_pause - self.elapsed_pause_time
                    hours = int(elapsed_time // 3600)
                    minutes = int((elapsed_time % 3600) // 60)
                    seconds = int(elapsed_time % 60)
                    time_text = f'Time: {hours:02}:{minutes:02}:{seconds:02}'
                    cv2.putText(img, time_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Draw rectangles around detected faces
                for (x, y, w, h) in faces:
                    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

                cv2.imshow('img', img)

            # Handle manual pause (Shift + W + Ctrl)
            if keyboard.is_pressed('shift+w+ctrl'):  # Pause manually
                if not self.manual_pause:
                    if self.is_running:
                        # Save elapsed time before manual pause
                        self.elapsed_time_before_pause += time() - self.start_time
                    self.is_running = False
                    self.manual_pause = True
                    self.pause_start_time = time()
                    self.cap.release()
                    cv2.destroyWindow('img')

            # Handle manual resume (Shift + Q + Ctrl)
            elif keyboard.is_pressed('shift+q+ctrl'):  # Resume manually
                if self.manual_pause:
                    self.cap = cv2.VideoCapture(0)
                    self.manual_pause = False
                    self.start_time = time()
                    self.is_running = True
                    self.last_check_time = time()

            # Stop if escape key is pressed
            if cv2.waitKey(1) & 0xFF == 27:
                break

        if self.cap.isOpened():
            self.cap.release()
        cv2.destroyAllWindows()


# Instantiate the CodeArea class and start the scan
if __name__ == "__main__":
    code = CodeArea()
    code.start_scan()

In [20]:
cv2.VideoCapture(0).release()